In [1]:
import spotipy
from spotipy . oauth2 import SpotifyClientCredentials
import networkx as nx
import pandas as pd

In [2]:
CLIENT_ID = "421a05c9619c4faaa5306c36a0bc48ca"
CLIENT_SECRET = "c89d362355544ea597cd918da2df93b5"

auth_manager = SpotifyClientCredentials (
client_id = CLIENT_ID ,
client_secret = CLIENT_SECRET )
sp = spotipy . Spotify ( auth_manager = auth_manager )

In [4]:

playlists = sp . user_playlists ("spotify")

while playlists :
    for i , playlist in enumerate ( playlists ["items"]) :
        print ("%4d %s %s" % ( i + 1 + playlists ["offset"] , playlist ["uri"] ,
    playlist ["name"]) )
    if playlists ["next"]:
        playlists = sp . next ( playlists )
    else :
        playlists = None

   1 spotify:playlist:37i9dQZF1DXcBWIGoYBM5M Today's Top Hits 
   2 spotify:playlist:37i9dQZF1DX0XUsuxWHRQd RapCaviar
   3 spotify:playlist:37i9dQZF1DX1lVhptIYRda Hot Country
   4 spotify:playlist:37i9dQZF1DX10zKzsJ2jva Viva Latino
   5 spotify:playlist:37i9dQZF1DX4JAvHpjipBk New Music Friday
   6 spotify:playlist:37i9dQZF1DX4sWSpwq3LiO Peaceful Piano
   7 spotify:playlist:37i9dQZF1DX4SBhb3fqCJd Are & Be
   8 spotify:playlist:37i9dQZF1DWXRqgorJj26U Rock Classics
   9 spotify:playlist:37i9dQZF1DX4dyzvuaRJ0n mint
  10 spotify:playlist:37i9dQZF1DXcF6B6QPhFDv Rock This
  11 spotify:playlist:37i9dQZF1DWXJfnUiYjUKT New Music Friday
  12 spotify:playlist:37i9dQZF1DXcRXFNfZr7Tp just hits
  13 spotify:playlist:37i9dQZF1DX4o1oenSJRJd All Out 2000s
  14 spotify:playlist:37i9dQZF1DXbTxeAdrVG2l All Out 90s
  15 spotify:playlist:37i9dQZF1DX4UtSsGT1Sbe All Out 80s
  16 spotify:playlist:37i9dQZF1DWTJ7xPn4vNaz All Out 70s
  17 spotify:playlist:37i9dQZF1DXaKIA8E7WcJj All Out 60s
  18 spotify:playlist:37

- Functions

1. Returns the ID of the artist.

In [5]:
def search_artist ( artist_name : str ) -> str :

    """
    Search for an artist in Spotify.

    :param artist_name: name to search for.
    :return: spotify artist id.

    """

    results = sp.search(q=artist_name, type="artist", limit=1)

    if results["artists"]["items"]:
        artist_id = results["artists"]["items"][0]["id"]
        return artist_id
    else:
        return ""

In [6]:
#An example for Skapiens:

artist = "ABBA"
print(artist, "ID:", search_artist(artist))

ABBA ID: 0LcJLqbBmaGUft1e9Mm8HV


2. Function

In [7]:
def crawler(seed: str, max_nodes_to_crawl: int, strategy: str = "BFS", out_filename: str = "g.graphml",return_last_cr = False) -> nx.DiGraph:
    """
    Crawl the Spotify artist graph, following related artists.

    :param seed: starting artist id.
    :param max_nodes_to_crawl: maximum number of nodes to crawl.
    :param strategy: BFS or DFS.
    :param out_filename: name of the graphml output file.
    :return: networkx directed graph.
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    G=nx.DiGraph()
    
    current_seed = seed
    crawled = [] 
    
    #first crawl
    found = sp.artist_related_artists(current_seed)
    crawled.append(current_seed)

    found_ids = [art["id"] for art in found["artists"]] #get found artists ids
    not_crawled = [id for id in found_ids if id not in crawled]
    tocrawl = not_crawled

    while len(G.nodes()) < max_nodes_to_crawl:
        
        node_pairs = [(current_seed,id) for id in found_ids]  #create node pairs with current crawled node
        G.add_edges_from(node_pairs) #add found edges and new nodes to graph

        if strategy == "DFS":
            #tocrawl = not_crawled #this filters already crawled nodes
            current_seed = not_crawled[0]

        elif strategy == "BFS":
            if len(tocrawl) == 0:
                tocrawl = not_crawled#this filters already crawled nodes
            
            current_seed = tocrawl[0]
            tocrawl.remove(tocrawl[0])

     
        #crawl next node
        found = sp.artist_related_artists(current_seed)
        crawled.append(current_seed)
        found_ids = [art["id"] for art in found["artists"]] #get found artists ids
        not_crawled = [id for id in found_ids if id not in crawled]
        
        if len(not_crawled) == 0: #if all found ids are already crawled
            print("All nodes crawled! Looking for a new one")
            while len(not_crawled) == 0:
                found = sp.artist_related_artists(current_seed)
                crawled.append(current_seed)
                found_ids = [art["id"] for art in found["artists"]] #get found artists ids
                not_crawled = [id for id in found_ids if id not in crawled]
            
    
    #save graph to file
    nx.write_graphml(G,out_filename)
    
    #depending on the output mode
    if return_last_cr:
        return G, crawled[-1]
    
    return G

In [8]:
graph = crawler(seed=search_artist("abba"), max_nodes_to_crawl=20)

In [9]:
graphs = []
graphs.append(graph)

3. Function

In [10]:
def get_track_data(graphs: list, out_filename: str) -> pd.DataFrame:

    """
    Get track data for each visited artist in the graph.

    :param graphs: a list of graphs with artists as nodes.
    :param out_filename: name of the csv output file.
    :return: pandas dataframe with track data.
    
    """
    data = []

    for graph in graphs:
        for artist_id in graph.nodes:
            
            artist = sp.artist(artist_id)
            #artist_id = search_artist(artist)
            top_tracks = sp.artist_top_tracks(artist_id,country='ES')


            for track in top_tracks["tracks"]:
                        
                track_info = sp.audio_features(track["id"])
                
                data.append({
                            
                        #Artist data
                              
                        "Artist": artist["name"],
                        "Artist_id" : artist_id,

                        #Basic Song Data

                        "Track Name": track["name"],
                        "Track ID": track["id"],
                        "Duration" :track["duration_ms"],
                        "Popularity": track["popularity"],

                        # Audio feature data for the song
                        "Danceablility": track_info[0]['danceability'],
                        "Energy": track_info[0]['energy'],
                        "Loudness": track_info[0]['loudness'],
                        "Speechiness": track_info[0]['speechiness'],
                        "Acoustincness": track_info[0]['acousticness'],
                        "Instrumentalness": track_info[0]['instrumentalness'],
                        "Liveness": track_info[0]['liveness'],
                        "Valence": track_info[0]['valence'],
                        "Tempo": track_info[0]['tempo'],

                        # Album data
                        "Album Name": track["album"]["name"],
                        "Album ID": track["album"]["id"],
                        "Album release date": track["album"]["release_date"]

                })
    

    df = pd.DataFrame(data)
    df.to_csv(out_filename, index=False) #df to csv
    
    return df      


In [11]:
get_track_data(graphs, out_filename="top_artists_tracks.csv")

,Artist,Artist_id,Track Name,Track ID,Duration,Popularity,Danceablility,Energy,Loudness,Speechiness,Acoustincness,Instrumentalness,Liveness,Valence,Tempo,Album Name,Album ID,Album release date
0,ABBA,0LcJLqbBmaGUft1e9Mm8HV,Dancing Queen,0GjEhVFGZW8afUYGChu3Rr,230400,85,0.543,0.870,-6.514,0.0428,0.3580,0.000939,0.7920,0.754,100.804,Arrival,1V6a99EbTTIegOhWoPxYI9,1976
1,ABBA,0LcJLqbBmaGUft1e9Mm8HV,Gimme! Gimme! Gimme! (A Man After Midnight),3vkQ5DAB1qQMYO4Mr9zJN6,292613,83,0.749,0.491,-9.655,0.0403,0.0200,0.008990,0.1590,0.536,119.528,Voulez-Vous,7iLuHJkrb9KHPkMgddYigh,1979
2,ABBA,0LcJLqbBmaGUft1e9Mm8HV,Lay All Your Love On Me,4euAGZTszWPrriggYK0HG9,274466,78,0.678,0.727,-7.844,0.0258,0.1170,0.003960,0.3540,0.710,133.481,Super Trouper,3ZdkT5buYFi1WQaB0XNNtf,1980
3,ABBA,0LcJLqbBmaGUft1e9Mm8HV,Mamma Mia,2TxCwUlqaOH3TIyJqGgR91,213266,80,0.749,0.748,-7.011,0.0323,0.2950,0.000443,0.4690,0.827,137.547,Abba,1kM6xcSYO5ASJaWgygznL7,1975
4,ABBA,0LcJLqbBmaGUft1e9Mm8HV,"Money, Money, Money",3lKN03PDeBnXZS7sDukRBt,186426,75,0.787,0.704,-6.873,0.0385,0.5350,0.003050,0.0890,0.646,120.701,Arrival,1V6a99EbTTIegOhWoPxYI9,1976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Elton John,3PhoLpVuITZKcymswpck5b,Goodbye Yellow Brick Road - Remastered 2014,4IRHwIZHzlHT1FQpRa5RdE,192826,81,0.559,0.473,-7.712,0.0279,0.4460,0.001410,0.1540,0.397,121.227,Goodbye Yellow Brick Road (Remastered),5WupqgR68HfuHt3BMJtgun,1973-10-05
206,Elton John,3PhoLpVuITZKcymswpck5b,Don't Go Breaking My Heart,7HW5WIw7ZgZORCzUxv5gW5,275440,81,0.729,0.844,-8.824,0.0344,0.1490,0.000003,0.0604,0.777,131.459,Rock Of The Westies,4ObsEU7mIXbo2n9A31yfds,1975-10-24
207,Elton John,3PhoLpVuITZKcymswpck5b,Tiny Dancer,2TVxnKdb3tqe1nhQWwwZCO,377093,79,0.414,0.428,-11.097,0.0278,0.3820,0.000243,0.1480,0.282,145.075,Madman Across The Water,2OZbaW9tgO62ndm375lFZr,1971-11-05
208,Elton John,3PhoLpVuITZKcymswpck5b,Bennie And The Jets - Remastered 2014,5Wj1rJnCLpMHdLaxsFtJLs,322548,76,0.665,0.673,-7.950,0.0493,0.2590,0.000008,0.4970,0.779,132.642,Goodbye Yellow Brick Road (Remastered),5WupqgR68HfuHt3BMJtgun,1973-10-05


4. Question

In [12]:
artist_id = search_artist("Drake")

gb = crawler(seed =artist_id, max_nodes_to_crawl= 200, strategy = "BFS", out_filename= "gb.graphml")
gd =  crawler(seed = artist_id, max_nodes_to_crawl= 200, strategy = "DFS", out_filename= "gd.graphml")


In [13]:
graphs = [gb]
get_track_data(graphs, out_filename="top_artists_tracks.csv")

,Artist,Artist_id,Track Name,Track ID,Duration,Popularity,Danceablility,Energy,Loudness,Speechiness,Acoustincness,Instrumentalness,Liveness,Valence,Tempo,Album Name,Album ID,Album release date
0,Drake,3TVXtAsR1Inumwj472S9r4,Search & Rescue,7aRCf5cLOFN1U7kvtChY1G,272112,91,0.817,0.440,-8.482,0.0734,0.060300,0.000001,0.3300,0.544,142.024,Search & Rescue,67WUCOjyl44d8fAFzHONjW,2023-04-07
1,Drake,3TVXtAsR1Inumwj472S9r4,Rich Flex,1bDbXMyjaUIooNwFE9wn0N,239359,90,0.561,0.520,-9.342,0.2440,0.050300,0.000002,0.3550,0.424,153.150,Her Loss,5MS3MvWHJ3lOZPLiMxzOU6,2022-11-04
2,Drake,3TVXtAsR1Inumwj472S9r4,Jimmy Cooks (feat. 21 Savage),3F5CgOj3wFlRv51JsHbxhe,218364,89,0.529,0.673,-4.711,0.1750,0.000307,0.000002,0.0930,0.366,165.921,"Honestly, Nevermind",3cf4iSSKd8ffTncbtKljXw,2022-06-17
3,Drake,3TVXtAsR1Inumwj472S9r4,One Dance,1zi7xx7UVEFkmKfv06H8x0,173986,88,0.792,0.625,-5.609,0.0536,0.007760,0.001800,0.3290,0.370,103.967,Views,40GMAhriYJRO1rsY4YdrZb,2016-05-06
4,Drake,3TVXtAsR1Inumwj472S9r4,WAIT FOR U (feat. Drake & Tems),59nOXPmaKlBfGMDeOVGrIK,189893,87,0.463,0.642,-4.474,0.3400,0.314000,0.000000,0.0686,0.339,83.389,I NEVER LIKED YOU,6tE9Dnp2zInFij4jKssysL,2022-04-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,Lil Jon & The East Side Boyz,3ciRvbBIVz9fBoPbtSYq4x,Real N***A Roll Call,3HnhrVBybVBzp9yEQcAzfC,308853,51,0.769,0.818,-4.724,0.2660,0.025800,0.000000,0.8450,0.786,84.028,Crunk Juice,0hk2hXNB5d65F400dhcdcV,2004-11-16
2015,Lil Jon & The East Side Boyz,3ciRvbBIVz9fBoPbtSYq4x,Bia' Bia' - Remix,1Un1dsZkA9cfZMLOXOyUBc,302106,47,0.768,0.700,-8.021,0.3050,0.001850,0.000000,0.0901,0.659,79.996,We Still Crunk!,3t1KcEIjoOo2W5oA1wAh1N,2000-08-15
2016,Lil Jon & The East Side Boyz,3ciRvbBIVz9fBoPbtSYq4x,Da Blow,3eW3brBXFNI992RsCy3cRo,359640,46,0.850,0.876,-4.583,0.1670,0.000845,0.000078,0.1240,0.626,138.493,Crunk Juice,0hk2hXNB5d65F400dhcdcV,2004-11-16
2017,Lil Jon & The East Side Boyz,3ciRvbBIVz9fBoPbtSYq4x,Get Low Remix,2HJB0UGH3wzPivgZjnohQG,311600,46,0.503,0.638,-5.851,0.1320,0.031200,0.000000,0.1220,0.196,108.322,Part II,1iJLCktIjEG4UwTterB9gw,2003-10-01


In [14]:
artist_id = search_artist("French Montana")
hb = crawler(seed =artist_id, max_nodes_to_crawl= 200, strategy = "BFS", out_filename= "gb.graphml")
